In [1]:
from importlib.machinery import SourceFileLoader
import pandas as pd
import numpy  as np

wr = SourceFileLoader("mobius", r"../../mobius.py").load_module()
cu = SourceFileLoader("mobius_calib_uncert_lmfit", r"../../mobius_calib_uncert_lmfit.py").load_module()

multi = SourceFileLoader("multiloop", r"../multiloop.py").load_module()
magicopt = SourceFileLoader("magicopt",  r"../magicopt.py").load_module()

In [2]:
wr.initialize('../../../Applications/MAGIC/magic_simple.dll')

parfile = 'templateparameters.dat'
infile  = 'templateinputs.dat'

dataset = wr.DataSet.setup_from_parameter_and_input_files(parfile, infile)

In [3]:
idx = ('Soil', 'Lake')

In [4]:
def do_optim(ds, ID, soil, depo, lake, printout=True) :

    ds.set_parameter_double('Initial exchangeable Ca on soil as % of CEC', ['Soil'], soil['ExCa'])
    ds.set_parameter_double('Initial exchangeable Mg on soil as % of CEC', ['Soil'], soil['ExMg'])
    ds.set_parameter_double('Initial exchangeable Na on soil as % of CEC', ['Soil'], soil['ExNa'])
    ds.set_parameter_double('Initial exchangeable K on soil as % of CEC', ['Soil'], soil['ExK'])
    
    
    params_a, comparisons_a = magicopt.get_acid_anion_setup(ds, idx, 'Observed')
    
    res_pars1, nfev1 = magicopt.calib(ds, params_a, comparisons_a, method='leastsq')
    
    ds.set_parameter_double('Initial exchangeable Ca on soil as % of CEC', ['Soil'], soil['ECa-0'])
    ds.set_parameter_double('Initial exchangeable Mg on soil as % of CEC', ['Soil'], soil['EMg-0'])
    ds.set_parameter_double('Initial exchangeable Na on soil as % of CEC', ['Soil'], soil['ENa-0'])
    ds.set_parameter_double('Initial exchangeable K on soil as % of CEC', ['Soil'], soil['EK-0'])
    
    params, comparisons = magicopt.get_base_cations_setup(ds, 
                            idx, 'Observed', usemax=True, obsE=True)
    
    #params.pretty_print()
    #Set initial guess to the present-day value
    params['ECa_S'].value = soil['ExCa']+20
    params['EMg_S'].value = soil['ExMg']
    params['ENa_S'].value = soil['ExNa']
    params['EK_S'].value = soil['ExK']
    
    params['WCa_S'].value*=0.2
    params['WMg_S'].value*=0.2
    params['WNa_S'].value*=0.2
    params['WK_S'].value*=0.2
    
    params['WK_S'].min = max(0.0, soil['UptK'] - depo['K'])
    
    
    ntries = 0
    err=0
    while (np.abs(err)>1 and ntries < 10) or ntries==0 :
        if err < 0. :
            params['WCa_S'].value*=2
            if ID==3096:
                params['ECa_S'].value+=5
        elif err > 0.:
            params['WCa_S'].value/=2
            if ID==3096:
                params['ECa_S'].value-=5
                
        res_pars2, nfev = magicopt.calib(ds, params, comparisons, 
                                    method='leastsq'
                                    )
        
        sim = ds.get_result_series('Ca(2+) ionic concentration', ['Lake'])
        obs = ds.get_input_series('Observed Ca', [], alignwithresults=True)
        err = np.nansum(sim-obs)
        
        ntries += 1
    
    #if np.abs(err)>1.:
    #    high_error_ids.append((ID,err))
    #    print('High Ca error (abs>1)')

    if printout :
        comp = comparisons
        comp += comparisons_a
        comp += [('SO4(2-) ionic concentration', ['Lake'], 'Observed SO4', [])]

        for comparison in comp :
            simname, simindexes, obsname, obsindexes, *_ = comparison

            sim = ds.get_result_series(simname, simindexes)
            obs = ds.get_input_series(obsname, obsindexes, alignwithresults=True)

            err = np.nansum(sim-obs)

            print('Error of %s: (sim-obs): %f' % (obsname, err))

        print('\n')
        res_pars1.pretty_print()
        res_pars2.pretty_print()
        print('\n')
        
        ds.write_parameters_to_file('MobiusFiles/params_%d.dat'%ID)
        ds.write_inputs_to_file('MobiusFiles/inputs_%d.dat'%ID)
    
    return res_pars1, res_pars2


def do_fuzzy(ds, ID, soil, depo, lake):
    
    nruns = 10
    
    fuzzy_params = [
        ('Observed NO3', [], 0.9, 1.1, 'obs'),
        ('Observed Ca', [], 0.9, 1.1, 'obs'),
        ('Observed Mg', [], 0.9, 1.1, 'obs'),
        ('Observed Na', [], 0.9, 1.1, 'obs'),
        ('Observed K', [], 0.9, 1.1, 'obs'),
        ('Depth', ['Soil'], 0.9, 1.1, 'par'),
    ]
    
    multipliers = np.zeros((len(fuzzy_params), nruns))
    values      = np.zeros((len(fuzzy_params), nruns))
    
    respars1 = []
    respars2 = []
    
    for idx, par in enumerate(fuzzy_params) :
        name, index, minimum, maximum, partype = par
        multipliers[idx, :] = np.random.uniform(minimum, maximum, nruns)
        
    for run in range(nruns) :
        ds2 = ds.copy()
        for idx, par in enumerate(fuzzy_params):
            name, index, minimum, maximum, partype = par
            if partype == 'obs':
                values = ds2.get_input_series(name, index)
                values *= multipliers[idx, run]
                ds2.set_input_series(name, index, values)
            elif partype == 'par':
                value = ds2.get_parameter_double(name, index)
                value *= multipliers[idx, run]
                ds2.set_parameter_double(name, index, value)
        res1,res2 = do_optim(ds2, ID, soil, depo, lake, printout=True)
        
        respars1.append(res1)
        respars2.append(res2)
        
        ds2.delete()
    
    #TODO: We need to store them somehow instead
    #return fuzzy_params, multipliers, respars1, respars2

In [ ]:
excelfile = 'CCE-%s BJC.xls'

#multi.do_magic_loop(dataset, excelfile, do_fuzzy, do_id=9)
multi.do_magic_loop(dataset, excelfile, do_optim)#, do_id=9)


#print('IDs with error in Ca > 1: ', high_error_ids)

Running lake Hokksjøen (ID 9)

Error of Observed Ca: (sim-obs): 0.000000
Error of Observed Mg: (sim-obs): -0.000000
Error of Observed Na: (sim-obs): -0.000000
Error of Observed K: (sim-obs): -0.000000
Error of Observed ECa: (sim-obs): 0.000000
Error of Observed EMg: (sim-obs): -0.000000
Error of Observed ENa: (sim-obs): -0.000000
Error of Observed EK: (sim-obs): -0.000000
Error of Observed NO3: (sim-obs): 0.000000
Error of Observed SO4: (sim-obs): 1.835064


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S    -99.14     -100        0 5.171e-15     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     25.51     0.01    88.54 3.707e-14     True     None     None
EK_S      5.967     0.01       15 8.214e-15     True     None     None
EMg_S     11.54     0.01    36.41 1.587e-14     True     None     None
ENa_S      5.45     0.01    13.08 8.093e-15     True     None     None
WCa_S     16.45        0    18

Error of Observed Ca: (sim-obs): 0.000004
Error of Observed Mg: (sim-obs): 0.000001
Error of Observed Na: (sim-obs): 0.000026
Error of Observed K: (sim-obs): 0.000000
Error of Observed ECa: (sim-obs): 0.000004
Error of Observed EMg: (sim-obs): 0.000001
Error of Observed ENa: (sim-obs): 0.000002
Error of Observed EK: (sim-obs): 0.000000
Error of Observed NO3: (sim-obs): 4.978401
Error of Observed SO4: (sim-obs): 1.040744


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0    18.02     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S      17.9     0.01    84.58 1.82e-05     True     None     None
EK_S      2.439     0.01    9.797 2.503e-06     True     None     None
EMg_S     3.639     0.01    24.07 3.69e-06     True     None     None
ENa_S     1.951     0.01    8.459 1.953e-06     True     None     None
WCa_S      12.4        0    171.3 2.436e-05     True     None     None

Error of Observed Ca: (sim-obs): 0.000011
Error of Observed Mg: (sim-obs): 0.000002
Error of Observed Na: (sim-obs): 0.000000
Error of Observed K: (sim-obs): 0.000000
Error of Observed ECa: (sim-obs): 0.000001
Error of Observed EMg: (sim-obs): 0.000000
Error of Observed ENa: (sim-obs): -0.000000
Error of Observed EK: (sim-obs): 0.000000
Error of Observed NO3: (sim-obs): 3.903287
Error of Observed SO4: (sim-obs): 3.817403


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0    7.238     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     17.23     0.01    73.16 2.246e-06     True     None     None
EK_S      4.325     0.01    13.45 5.845e-07     True     None     None
EMg_S     4.719     0.01    26.15 6.208e-07     True     None     None
ENa_S     1.379     0.01    7.359 1.868e-07     True     None     None
WCa_S     27.26        0    194.4 1.264e-05     True     None     N

Error of Observed Ca: (sim-obs): 0.000000
Error of Observed Mg: (sim-obs): 0.000000
Error of Observed Na: (sim-obs): 0.000000
Error of Observed K: (sim-obs): -0.000000
Error of Observed ECa: (sim-obs): 0.000000
Error of Observed EMg: (sim-obs): 0.000000
Error of Observed ENa: (sim-obs): 0.000000
Error of Observed EK: (sim-obs): -0.000000
Error of Observed NO3: (sim-obs): 1.681877
Error of Observed SO4: (sim-obs): -0.023657


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0    2.996     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     32.44     0.01       90 1.085e-13     True     None     None
EK_S      4.232     0.01    12.23 1.547e-14     True     None     None
EMg_S     8.209     0.01    31.63 3.031e-14     True     None     None
ENa_S     4.086     0.01    11.74 1.617e-14     True     None     None
WCa_S        43        0    212.4 5.738e-13     True     None    

Error of Observed Ca: (sim-obs): 0.000000
Error of Observed Mg: (sim-obs): -0.000000
Error of Observed Na: (sim-obs): 0.000000
Error of Observed K: (sim-obs): 0.000000
Error of Observed ECa: (sim-obs): 0.000000
Error of Observed EMg: (sim-obs): 0.000000
Error of Observed ENa: (sim-obs): -0.000000
Error of Observed EK: (sim-obs): 0.000000
Error of Observed NO3: (sim-obs): 0.822641
Error of Observed SO4: (sim-obs): 0.274121


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0   0.7084     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     32.59     0.01       90 2.148e-13     True     None     None
EK_S       4.49     0.01    12.59 3.427e-14     True     None     None
EMg_S     17.38     0.01       50 1.246e-13     True     None     None
ENa_S     5.581     0.01    14.09 4.55e-14     True     None     None
WCa_S     76.77        0      265 2.278e-12     True     None     N

...

Error of Observed Ca: (sim-obs): -0.000048
Error of Observed Mg: (sim-obs): -0.000201
Error of Observed Na: (sim-obs): -0.000889
Error of Observed K: (sim-obs): -0.000004
Error of Observed ECa: (sim-obs): -0.000029
Error of Observed EMg: (sim-obs): -0.000049
Error of Observed ENa: (sim-obs): -0.000027
Error of Observed EK: (sim-obs): -0.000004
Error of Observed NO3: (sim-obs): 1.229817
Error of Observed SO4: (sim-obs): 0.142168


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0     None     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     20.52     0.01    82.94 0.0002091     True     None     None
EK_S      3.928     0.01     12.2 4.156e-05     True     None     None
EMg_S      7.49     0.01    33.48 7.896e-05     True     None     None
ENa_S     4.897     0.01    13.74 5.397e-05     True     None     None
WCa_S      39.4        0    222.5  0.00178     True     None

Error of Observed Ca: (sim-obs): -0.000070
Error of Observed Mg: (sim-obs): 0.000749
Error of Observed Na: (sim-obs): -0.000200
Error of Observed K: (sim-obs): 0.008691
Error of Observed ECa: (sim-obs): -0.001178
Error of Observed EMg: (sim-obs): 0.000191
Error of Observed ENa: (sim-obs): 0.000018
Error of Observed EK: (sim-obs): -0.000158
Error of Observed NO3: (sim-obs): 1.479787
Error of Observed SO4: (sim-obs): -0.090714


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0     None     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     12.24     0.01     61.8  0.01603     True     None     None
EK_S      2.557     0.01       10 0.003395     True     None     None
EMg_S     4.175     0.01     24.4 0.005449     True     None     None
ENa_S     3.567     0.01     11.6 0.004859     True     None     None
WCa_S     4.402        0    164.2  0.03871     True     None     No

Error of Observed Ca: (sim-obs): -0.000000
Error of Observed Mg: (sim-obs): 0.000000
Error of Observed Na: (sim-obs): 0.000000
Error of Observed K: (sim-obs): -0.000000
Error of Observed ECa: (sim-obs): -0.000000
Error of Observed EMg: (sim-obs): 0.000000
Error of Observed ENa: (sim-obs): 0.000000
Error of Observed EK: (sim-obs): -0.000000
Error of Observed NO3: (sim-obs): 0.000000
Error of Observed SO4: (sim-obs): -0.020578


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S    -68.76     -100        0 3.36e-14     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     4.548     0.01     30.2 1.691e-14     True     None     None
EK_S      2.378     0.01      9.4 8.76e-15     True     None     None
EMg_S     3.101     0.01     20.4 1.182e-14     True     None     None
ENa_S     2.154     0.01        9 8.161e-15     True     None     None
WCa_S     32.42        0    209.2 4.961e-13     True     None   

Error of Observed Ca: (sim-obs): -0.007680
Error of Observed Mg: (sim-obs): 0.003429
Error of Observed Na: (sim-obs): 0.001804
Error of Observed K: (sim-obs): 0.000052
Error of Observed ECa: (sim-obs): 0.000167
Error of Observed EMg: (sim-obs): 0.000094
Error of Observed ENa: (sim-obs): -0.000001
Error of Observed EK: (sim-obs): -0.000005
Error of Observed NO3: (sim-obs): -0.000000
Error of Observed SO4: (sim-obs): 23.196100


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S    -73.04     -100        0 4.43e-14     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     40.22     0.01       90 0.0008679     True     None     None
EK_S      6.673     0.01       15 0.0001291     True     None     None
EMg_S     20.19     0.01    49.52 0.0005071     True     None     None
ENa_S     2.391     0.01    7.877 5.069e-05     True     None     None
WCa_S     50.55        0    258.2 0.004209     True     None   

Error of Observed Ca: (sim-obs): 0.388899
Error of Observed Mg: (sim-obs): 0.031520
Error of Observed Na: (sim-obs): 0.035504
Error of Observed K: (sim-obs): 0.025114
Error of Observed ECa: (sim-obs): -0.045806
Error of Observed EMg: (sim-obs): -0.010953
Error of Observed ENa: (sim-obs): -0.003258
Error of Observed EK: (sim-obs): -0.003367
Error of Observed NO3: (sim-obs): -0.000000
Error of Observed SO4: (sim-obs): 23.029816


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S    -60.47     -100        0 1.08e-14     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     21.32     0.01    65.36   0.1684     True     None     None
EK_S      2.895     0.01    9.597  0.01822     True     None     None
EMg_S     5.714     0.01    19.57    0.038     True     None     None
ENa_S     2.465     0.01    8.085  0.01536     True     None     None
WCa_S     88.34        0    340.7    1.932     True     None     N